# week4 授業課題1


## 【問題1】コンペティション内容の確認  

### コンペティションのOverviewページ読み、「Home Credit Default Risk」について以下の観点について確認してください。

- 何を学習し、何を予測するのか<br>
 application_train.csvの特徴量を学習させ、TARGET変数の確率を予測す
- どのようなファイルを作りKaggleに提出するか<br>
提出ファイル テストセット内のSK_ID_CURRごとに、TARGET変数の確率を予測する必要があります。ファイルにはヘッダーが含まれ、次の形式である必要があります。  
samplesubmission.csvと同様の形式で提出する。<br>
>SK_ID_CURR,TARGET
>100001,0.1
>100005,0.9
>100013,0.2
>etc.
↓↓↓
100001,1
>- 提出されたものはどういった指標値で評価されるのか
    - 提出物は、予測された確率と観測されたターゲットの間のROC曲線の下の領域で評価されます。ROC曲線は、さまざまなしきい値設定での偽陽性率（FPR）に対して真陽性率（TPR）をプロットすることによって作成されます。  
    ローンが返済されない確率が知りたいので２列目（）  
>Make predictions  
>Make sure to select the second column only  
>log_reg_pred = log_reg.predict_proba(test)[:, 1]  
># Submission dataframe  
>submit = app_test[['SK_ID_CURR']]   
>submit['TARGET'] = log_reg_pred  
>submit.head()  
>https://www.kaggle.com/willkoehrsen/start-here-a-gentle-introduction
>42セル参照

## 【問題2】学習と検証  

>データを簡単に分析、前処理し、学習、検証するまでの一連の流れを作成・実行してください。
>検証にはこのコンペティションで使用される評価指標を用いるようにしてください。学習に用いる手法は指定しません。

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns

In [2]:
train_df = pd.read_csv("application_train.csv")
test_df = pd.read_csv("application_test.csv")
df_copy = test_df.copy(deep=False)

In [3]:
train_df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# 学習データから目的変数を分ける
X_train_df = train_df.iloc[:, 2:]
y_train_df = train_df.iloc[:, 1:2]
print(X_train_df.shape)
print(y_train_df.shape)
# 学習データの説明変数とテストデータを結合
X_all_df = pd.concat([X_train_df, test_df]) 

(307511, 120)
(307511, 1)


/Users/arisa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


In [5]:
X_train_df.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,Unaccompanied,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,Unaccompanied,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
y_train_df.head()

,TARGET
0,1
1,0
2,0
3,0
4,0


In [7]:
# 学習に使う特徴量を絞る
index_array = X_all_df.columns
index_list = index_array.tolist()
index_list.remove("DAYS_BIRTH")
index_list.remove("EXT_SOURCE_1")
index_list.remove("EXT_SOURCE_2")
index_list.remove("EXT_SOURCE_3")
X_all_df.drop(columns=index_list, inplace=True)

In [8]:
X_all_df.shape

(356255, 4)

In [9]:
test_df.shape

(48744, 121)

In [10]:
train_df.shape

(307511, 122)

In [11]:
# 欠損値を確認する
X_all_df.isnull().sum()

DAYS_BIRTH           0
EXT_SOURCE_1    193910
EXT_SOURCE_2       668
EXT_SOURCE_3     69633
dtype: int64

In [12]:
# 欠損値を0で埋める
X_all_df.fillna(0, inplace=True)

In [13]:
X_all_df.isnull().sum()

DAYS_BIRTH      0
EXT_SOURCE_1    0
EXT_SOURCE_2    0
EXT_SOURCE_3    0
dtype: int64

In [14]:
# 結合データをtrainデータとtestデータに分ける
X_train_df = X_all_df.iloc[:train_df.shape[0], :]
X_test_df = X_all_df.iloc[train_df.shape[0]:, :]

In [15]:
X_train_df.shape

(307511, 4)

In [16]:
X_test_df.shape

(48744, 4)

In [17]:
y_train_df.shape

(307511, 1)

In [18]:
# testデータをndarrayに変換
X_test = X_test_df.values

In [19]:
# trainデータを訓練データ（train）75%、検証データ(var)25%で分割 後で検証するためシード値を0にする
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train_df, y_train_df, test_size=0.25, random_state=0)


In [20]:
X_train.shape,X_val.shape

((230633, 4), (76878, 4))

In [21]:
# Xのみ標準化
from sklearn.preprocessing import StandardScaler
scaler_x = StandardScaler()
scaler_x.fit(X_train)
X_train_std = scaler_x.transform(X_train)
X_val_std = scaler_x.transform(X_val)
X_test_std = scaler_x.transform(X_test)

In [22]:
# ランダムフォレスト
from sklearn.ensemble import RandomForestClassifier
regr = RandomForestClassifier()
regr.fit(X_train_std, y_train)
y_pred_randomF_val = regr.predict(X_val_std)
y_pred_proba_randomF_val = regr.predict_proba(X_val_std)
y_pred_proba_0_randomF_val = regr.predict_proba(X_val_std)[:, 0]
y_pred_proba_1_randomF_val = regr.predict_proba(X_val_std)[:, 1]

/Users/arisa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/Users/arisa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


predictとpredict_probaの挙動の違いについて、バリデーションデータを使いながら検証する

In [23]:
# predictは予測値　今回は２値問題なので0,1
np.unique(y_pred_randomF_val)

array([0, 1])

In [24]:
# predict_probaは0,1の確率
np.unique(y_pred_proba_randomF_val)

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [25]:
# predictは予測値　今回は２値問題なので0,1
count = 0
for val in y_pred_randomF_val:
    print(val)
    count += 1
    if count == 10:
        break

0
0
0
0
0
0
0
0
0
0


In [26]:
# predict_probaは0,1の確率それぞれを配列で渡す
count = 0
for val in y_pred_proba_randomF_val:
    print(val)
    count += 1
    if count == 10:
        break

[0.8 0.2]
[1. 0.]
[0.7 0.3]
[1. 0.]
[0.7 0.3]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]
[1. 0.]


In [27]:
# predict_proba(X)[:, 0]は列指定をしているので0の確率を渡す
count = 0
for val in y_pred_proba_0_randomF_val:
    print(val)
    count += 1
    if count == 10:
        break

0.8
1.0
0.7
1.0
0.7
1.0
1.0
1.0
1.0
1.0


In [28]:
# predict_proba(X)[:, 1]は列指定をしているので1の確率を渡す
count = 0
for val in y_pred_proba_1_randomF_val:
    print(val)
    count += 1
    if count == 10:
        break

0.2
0.0
0.3
0.0
0.3
0.0
0.0
0.0
0.0
0.0


In [29]:
# ROC AUC で評価
from sklearn.metrics import roc_auc_score
roc_auc_score(y_val, y_pred_proba_1_randomF_val)

0.6241771334337793

In [30]:
# バリデーションデータの正解率を確認
from sklearn.metrics import accuracy_score
accuracy_score(y_val, y_pred_randomF_val)

0.9145269127708837

## 【問題3】テストデータに対する推定

In [31]:
# テストデータの１である確率と分類結果
y_pred_randomF_test = regr.predict(X_test_std)

In [32]:
# 種類を確認
np.unique(y_pred_randomF_test)

array([0, 1])

In [33]:
y_pred_proba_1_randomF_test = regr.predict(X_test_std)

In [34]:
count = 0
for val in y_pred_proba_1_randomF_test:
    print(val)
    count += 1
    if count == 10:
        break

0
0
0
0
0
0
0
0
0
0


In [35]:
submission = pd.DataFrame({"SK_ID_CURR": df_copy["SK_ID_CURR"], "TARGET": y_pred_proba_1_randomF_test})
submission.to_csv("submissin_1.csv", index=False)

kaggleへ提出した。スコアは0.50892だった

## 【問題4】特徴量エンジニアリング

検証１：利用者は女性が多かったが、返済が予定通りできなかった人は男性が多かったことから性別を特徴量に追加してみる
CODE_GENDER（性別）はカテゴリデータなので、ラベルエンコーディング（one-hot）で置き換えて特徴量を追加する

In [36]:
# 学習データの説明変数とテストデータを結合
X_all_df = pd.concat([X_train_df, test_df]) 
# 学習に使う特徴量を絞る
index_array = X_all_df.columns
index_list = index_array.tolist()
index_list.remove("DAYS_BIRTH")
index_list.remove("CODE_GENDER")
index_list.remove("EXT_SOURCE_1")
index_list.remove("EXT_SOURCE_2")
index_list.remove("EXT_SOURCE_3")
X_all_df.drop(columns=index_list, inplace=True)

/Users/arisa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [37]:
def prepro(X_all_df, train_df):
    # 前処理用(バリデーションと標準化)の関数を作成
    # 結合データをtrainデータとtestデータに分ける
    X_train_df = X_all_df.iloc[:train_df.shape[0], :]
    X_test_df = X_all_df.iloc[train_df.shape[0]:, :]
    X_test = X_test_df.values
    # trainデータを訓練データ（train）75%、検証データ(var)25%で分割
    X_train, X_val, y_train, y_val = train_test_split(X_train_df, y_train_df, test_size=0.25, random_state=0)
    scaler_x = StandardScaler()
    # Xのみ標準化
    scaler_x.fit(X_train)
    X_train_std = scaler_x.transform(X_train)
    X_val_std = scaler_x.transform(X_val)
    X_test_std = scaler_x.transform(X_test)
    
    return X_train_std, X_val_std, y_train, y_val, X_test_std

In [38]:
# CODE_GENDERをone-hot
X_all_df = pd.get_dummies(X_all_df, columns=["CODE_GENDER"])
X_all_df.head()

,DAYS_BIRTH,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,CODE_GENDER_F,CODE_GENDER_M
0,-9461,0.083037,0.262949,0.139376,0,0
1,-16765,0.311267,0.622246,0.000000,0,0
2,-19046,0.000000,0.555912,0.729567,0,0
3,-19005,0.000000,0.650442,0.000000,0,0
4,-19932,0.000000,0.322738,0.000000,0,0


In [39]:
# 欠損値を確認する
X_all_df.isnull().sum()

DAYS_BIRTH           0
EXT_SOURCE_1     20532
EXT_SOURCE_2         8
EXT_SOURCE_3      8668
CODE_GENDER_F        0
CODE_GENDER_M        0
dtype: int64

In [40]:
# 欠損値を0で埋める
X_all_df.fillna(0, inplace=True)

In [41]:
#前処理(バリデーションと標準化)する
X_train_std, X_val_std, y_train, y_val, X_test_std = prepro(X_all_df, train_df)

In [42]:
# ランダムフォレスト
from sklearn.ensemble import RandomForestClassifier
regr = RandomForestClassifier()
regr.fit(X_train_std, y_train)
y_pred_randomF_val_1 = regr.predict(X_val_std)
y_pred_proba_randomF_val = regr.predict_proba(X_val_std)
y_pred_proba_1_randomF_val = regr.predict_proba(X_val_std)[:, 1]

/Users/arisa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


In [43]:
# バリデーションデータの1の確率を渡す
count = 0
for val in y_pred_randomF_val_1:
    print(val)
    count += 1
    if count == 10:
        break

0
0
0
0
0
0
0
0
0
0


In [44]:
# ROC AUC で評価
roc_auc_score(y_val, y_pred_proba_1_randomF_val)

0.6258631441993865

In [45]:
# バリデーションデータの正解率を確認
accuracy_score(y_val, y_pred_randomF_val_1)

0.9158536902624939

考察：正解率で比較すると0.9149→0.9153精度は多少上がった
ROC AUC で比較すると0.623→0.624

検証2：DAYS_BIRTHは契約時から遡った値のため、-1をかけて、年単位（/365）にする

In [46]:
# 学習データの説明変数とテストデータを結合
X_all_df = pd.concat([X_train_df, test_df]) 
# 学習に使う特徴量を絞る
index_array = X_all_df.columns
index_list = index_array.tolist()
index_list.remove("DAYS_BIRTH")
index_list.remove("EXT_SOURCE_1")
index_list.remove("EXT_SOURCE_2")
index_list.remove("EXT_SOURCE_3")
X_all_df.drop(columns=index_list, inplace=True)

/Users/arisa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [47]:
X_all_df.head()

,DAYS_BIRTH,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3
0,-9461,0.083037,0.262949,0.139376
1,-16765,0.311267,0.622246,0.000000
2,-19046,0.000000,0.555912,0.729567
3,-19005,0.000000,0.650442,0.000000
4,-19932,0.000000,0.322738,0.000000


In [48]:
X_all_df["DAYS_BIRTH"] = X_all_df["DAYS_BIRTH"] *(-1)/365 

In [49]:
X_all_df.head()

,DAYS_BIRTH,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3
0,25.920548,0.083037,0.262949,0.139376
1,45.931507,0.311267,0.622246,0.000000
2,52.180822,0.000000,0.555912,0.729567
3,52.068493,0.000000,0.650442,0.000000
4,54.608219,0.000000,0.322738,0.000000


In [50]:
# 欠損値を0で埋める
X_all_df.fillna(0, inplace=True)

In [51]:
#前処理(バリデーションと標準化)する
X_train_std, X_val_std, y_train, y_val, X_test_std = prepro(X_all_df, train_df)

In [52]:
# ランダムフォレスト
regr = RandomForestClassifier()
regr.fit(X_train_std, y_train)
y_pred_randomF_val_2 = regr.predict(X_val_std)
y_pred_proba_randomF_val = regr.predict_proba(X_val_std)
y_pred_proba_1_randomF_val = regr.predict_proba(X_val_std)[:, 1]

/Users/arisa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [53]:
# バリデーションデータの1の確率を渡す
count = 0
for val in y_pred_randomF_val_2:
    print(val)
    count += 1
    if count == 10:
        break

0
0
0
0
0
0
0
0
0
0


In [54]:
# ROC AUC で評価
roc_auc_score(y_val, y_pred_proba_1_randomF_val)

0.6214285086582341

In [55]:
# バリデーションデータの正解率を確認
accuracy_score(y_val, y_pred_randomF_val_2)

0.9149431566898202

考察：正解率で比較すると0.9149→0.9152精度は多少上がった
ROC AUC 0.623→0.624

検証3：NAME_CONTRACT_TYPE(ローンの種類)をone-hotエンコーディングしてみる

In [56]:
# 学習データの説明変数とテストデータを結合
X_all_df = pd.concat([X_train_df, test_df]) 
# 学習に使う特徴量を絞る
index_array = X_all_df.columns
index_list = index_array.tolist()
index_list.remove("DAYS_BIRTH")
index_list.remove("NAME_CONTRACT_TYPE")
index_list.remove("EXT_SOURCE_1")
index_list.remove("EXT_SOURCE_2")
index_list.remove("EXT_SOURCE_3")
X_all_df.drop(columns=index_list, inplace=True)

/Users/arisa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [57]:
X_all_df = pd.get_dummies(X_all_df, columns=["NAME_CONTRACT_TYPE"])
X_all_df.head()

,DAYS_BIRTH,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans
0,-9461,0.083037,0.262949,0.139376,0,0
1,-16765,0.311267,0.622246,0.000000,0,0
2,-19046,0.000000,0.555912,0.729567,0,0
3,-19005,0.000000,0.650442,0.000000,0,0
4,-19932,0.000000,0.322738,0.000000,0,0


In [58]:
# 欠損値を0で埋める
X_all_df.fillna(0, inplace=True)
#前処理(バリデーションと標準化)する
X_train_std, X_val_std, y_train, y_val, X_test_std = prepro(X_all_df, train_df)
# ランダムフォレスト
regr = RandomForestClassifier()
regr.fit(X_train_std, y_train)
y_pred_randomF_val_3 = regr.predict(X_val_std)
y_pred_proba_randomF_val = regr.predict_proba(X_val_std)
y_pred_proba_1_randomF_val = regr.predict_proba(X_val_std)[:, 1]

/Users/arisa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys


In [59]:
# バリデーションデータの1の確率を渡す
count = 0
for val in y_pred_proba_1_randomF_val:
    print(val)
    count += 1
    if count == 10:
        break

0.5
0.2
0.0
0.0
0.1
0.1
0.1
0.0
0.1
0.0


In [60]:
# ROC AUC で評価
roc_auc_score(y_val, y_pred_proba_1_randomF_val)

0.6217255826574069

In [61]:
# バリデーションデータの正解率を確認
accuracy_score(y_val, y_pred_randomF_val_3)

0.9155935378131586

考察：正解率で比較すると0.9149→0.9154精度は多少上がった

検証４：モデルをロジスティック回帰にしてみる

In [62]:
# 学習データの説明変数とテストデータを結合
X_all_df = pd.concat([X_train_df, test_df]) 
# 学習に使う特徴量を絞る
index_array = X_all_df.columns
index_list = index_array.tolist()
index_list.remove("DAYS_BIRTH")
index_list.remove("EXT_SOURCE_1")
index_list.remove("EXT_SOURCE_2")
index_list.remove("EXT_SOURCE_3")
X_all_df.drop(columns=index_list, inplace=True)

/Users/arisa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [63]:
# 欠損値を0で埋める
X_all_df.fillna(0, inplace=True)
#前処理(バリデーションと標準化)する
X_train_std, X_val_std, y_train, y_val, X_test_std = prepro(X_all_df, train_df)
# ロジスティック回帰
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X_train_std, y_train)
y_pred_logistic_val = clf.predict(X_val_std)
y_pred_proba_logistic_val = clf.predict_proba(X_val_std)
y_pred_proba_1_logistic_val = clf.predict_proba(X_val_std)[:, 1]

/Users/arisa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [64]:
# バリデーションデータの1の確率を表示
count = 0
for val in y_pred_proba_1_logistic_val:
    print(val)
    count += 1
    if count == 10:
        break

0.11850333718630192
0.021764261612783484
0.1074430780413442
0.06062910127268814
0.17495905399529293
0.052752029219668546
0.04561919395901134
0.08650889440674109
0.033388819795136825
0.05849945533802529


In [65]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_val, y_pred_proba_1_logistic_val)

0.701192403718053

In [66]:
# バリデーションデータの正解率を確認
accuracy_score(y_val, y_pred_logistic_val)

0.9207705715549311

考察：正解率で比較すると0.9149→0.9207精度は0.1ほど上がった

検証5：検証1〜４を全て盛り込んでみる

In [67]:
# 学習データの説明変数とテストデータを結合
X_all_df = pd.concat([X_train_df, test_df]) 
# 学習に使う特徴量を絞る
index_array = X_all_df.columns
index_list = index_array.tolist()
index_list.remove("DAYS_BIRTH")
index_list.remove("CODE_GENDER")
index_list.remove("NAME_CONTRACT_TYPE")
index_list.remove("EXT_SOURCE_1")
index_list.remove("EXT_SOURCE_2")
index_list.remove("EXT_SOURCE_3")
X_all_df.drop(columns=index_list, inplace=True)

/Users/arisa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [68]:
# CODE_GENDER,NAME_CONTRACT_TYPEをone-hot
X_all_df = pd.get_dummies(X_all_df, columns=["CODE_GENDER", "NAME_CONTRACT_TYPE"])

In [69]:
X_all_df["DAYS_BIRTH"] = X_all_df["DAYS_BIRTH"] *(-1)/365 
X_all_df.head()

,DAYS_BIRTH,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,CODE_GENDER_F,CODE_GENDER_M,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans
0,25.920548,0.083037,0.262949,0.139376,0,0,0,0
1,45.931507,0.311267,0.622246,0.000000,0,0,0,0
2,52.180822,0.000000,0.555912,0.729567,0,0,0,0
3,52.068493,0.000000,0.650442,0.000000,0,0,0,0
4,54.608219,0.000000,0.322738,0.000000,0,0,0,0


In [70]:
# 欠損値を0で埋める
X_all_df.fillna(0, inplace=True)
#前処理(バリデーションと標準化)する
X_train_std, X_val_std, y_train, y_val, X_test_std = prepro(X_all_df, train_df)
# ロジスティック回帰
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X_train_std, y_train)
y_pred_logistic_val = clf.predict(X_val_std)
y_pred_proba_logistic_val = clf.predict_proba(X_val_std)
y_pred_proba_1_logistic_val = clf.predict_proba(X_val_std)[:, 1]

/Users/arisa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [71]:
# バリデーションデータの1の確率を表示
count = 0
for val in y_pred_proba_1_logistic_val:
    print(val)
    count += 1
    if count == 10:
        break

0.11850333718630188
0.021764261612783484
0.1074430780413442
0.06062910127268817
0.1749590539952929
0.052752029219668574
0.04561919395901136
0.08650889440674112
0.03338881979513685
0.058499455338025315


In [72]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_val, y_pred_proba_1_logistic_val)

0.701192403718053

In [73]:
# バリデーションデータの正解率を確認
accuracy_score(y_val, y_pred_logistic_val)

0.9207705715549311

考察：ロジスティック回帰では、今回追加した特徴量やエンコーディングをしても変化は見られなかった

検証6：年齢と相関の高かったEXT_SOURCE_1を抜いてみる

In [74]:
# 学習データの説明変数とテストデータを結合
X_all_df = pd.concat([X_train_df, test_df]) 
# 学習に使う特徴量を絞る
index_array = X_all_df.columns
index_list = index_array.tolist()
index_list.remove("DAYS_BIRTH")
index_list.remove("CODE_GENDER")
index_list.remove("NAME_CONTRACT_TYPE")
index_list.remove("EXT_SOURCE_2")
index_list.remove("EXT_SOURCE_3")
X_all_df.drop(columns=index_list, inplace=True)

/Users/arisa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [75]:
# CODE_GENDER,NAME_CONTRACT_TYPEをone-hot
X_all_df = pd.get_dummies(X_all_df, columns=["CODE_GENDER", "NAME_CONTRACT_TYPE"])

In [76]:
X_all_df["DAYS_BIRTH"] = X_all_df["DAYS_BIRTH"] *(-1)/365 

In [77]:
# 欠損値を0で埋める
X_all_df.fillna(0, inplace=True)
#前処理(バリデーションと標準化)する
X_train_std, X_val_std, y_train, y_val, X_test_std = prepro(X_all_df, train_df)
# ロジスティック回帰
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X_train_std, y_train)
y_pred_logistic_val = clf.predict(X_val_std)
y_pred_proba_logistic_val = clf.predict_proba(X_val_std)
y_pred_proba_1_logistic_val = clf.predict_proba(X_val_std)[:, 1]
y_pred_proba_1_logistic_test = clf.predict_proba(X_test_std)[:, 1]

/Users/arisa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [78]:
# バリデーションデータの1の確率を表示
count = 0
for val in y_pred_proba_1_logistic_val:
    print(val)
    count += 1
    if count == 10:
        break

0.15324909336995693
0.029825641926139766
0.09531064567755855
0.05074403937855775
0.153932247344368
0.04570184512268603
0.038835346219343175
0.07686604674696079
0.0286171830072577
0.05096711764318456


In [79]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_val, y_pred_proba_1_logistic_val)

0.6965302663744932

In [80]:
# バリデーションデータの正解率を確認
accuracy_score(y_val, y_pred_logistic_val)

0.9207705715549311

考察：predict_probaの値は変わったが、正解率、roc_auc_scoreともに変化がなかった

In [81]:
submission = pd.DataFrame({"SK_ID_CURR": df_copy["SK_ID_CURR"], "TARGET": y_pred_proba_1_logistic_test})
submission.to_csv("submission_3.csv", index=False)

kaggleに提出したところ、スコアが0.68014に下がってしまった。検証データでのroc_auc_scoreや正解率は上がったが、テストデータに対してはあまり良くなかったようだ。